In [7]:

import gymnasium as gym
import numpy as np
import json
import forex as fx

import ray
# ray.init(logging_level="info", log_to_driver=False)
ray.shutdown()
class deriv12_env(gym.Env):
    def __init__(self, config=None):
        # self.env = ForexApi(config["apiKey"], config["accountID"])

        self.observation_space = gym.spaces.Box(0.0, 2.0, (64,), np.float32)
        self.action_space = gym.spaces.Discrete(2)
        
        
        self.arrAll = fx.min("EUR_USD").to_numpy()[0][:,2]
        self.bal = 100
        self.episodeRewards = []
        self.stepRewards = 0
        self.info = {}
        # Actons : buy, sell, doNothing

    def reset(self, seed=None, options=None):
        
        self.bal = 100
        self.stepRewards = 0

        return np.zeros(64), {}

    def step(self, action):
        """
        the y variable y[0] is the deriv12 action that is found
        
        """
        done = False
        reward = 0.0

        s = np.random.randint(0,len(self.arrAll)- 129)
        y = self.arrAll[s:s+64]
        val = 0

        buy,sell,tbuy,tsell = fx.algo.deriv12(y,raw_data=True)

        algoAction = None
        groundTruth = y
        
        for i in range(len(buy)-1,1,-1):
            if buy[i] != 0:
                algoAction  = 1
                val = y[i]
                break
            elif sell[i] != 0:
                algoAction = -1
                val = y[i]
                break
        
        
        if self.bal <= 4 or self.bal > 500:
            self.info["episode_rewards"].append(self.stepRewards)
            print(f"Episode ended. Total reward: {self.stepRewards}")
            done = True

        
        f = self.arrAll[s+120]
        correctBuy = algoAction == 1 and f - val > 0
        correctSell = algoAction == -1 and f-val < 0
        incorrectBuy = algoAction == 1 and f-val < 0
        incorrectSell = algoAction == -1 and f-val > 0

        if done:
            self.info["episode_rewards"] = self.info["episode_rewards"].append(self.stepRewards)
            reward = 0.0
            return y, reward, done, False, self.info

        if action == 0: # Accept
            if abs(f-val) <= 0.001:
                
                self.stepRewards += -0.1
                reward = -0.01
            if correctBuy or correctSell:
                self.bal += 1
                self.stepRewards += 2
                reward = 2.0
            
            else:
                self.bal -= 1
                self.stepRewards -= 2
                reward = 2.0
                
        elif action == 1: # Decline 
            if incorrectBuy or incorrectSell:    
                self.bal += 1
                
                self.stepRewards += 2
                reward = 2.0
            else:
                self.bal -= 1 
                self.stepRewards += -2
                reward = -2.0
                
        self.stepRewards += reward  # Track cumulative episode rewards
        # Return next observation, reward, terminated, truncated, and info dict.
        return y, reward, done, False, self.info



In [6]:
import forex as fx
import yfinance as yf
data = yf.download("EURUSD=X", interval= "1h", progress=False)#.reset_index()
    


1 Failed download:
['EURUSD=X']: JSONDecodeError('Expecting value: line 1 column 1 (char 0)')


In [9]:



apiKey = None
accountID = None
with open("dev_settings.json", "r") as file:
    d = json.load(file)
    accountID = d["acckey"]
    apiKey = d["accid"]

from ray.tune.registry import register_env

env = deriv12_env
def env_creator(env_config):
    return env(env_config)  # Pass the config to your environment

register_env("Deriv12Env", env_creator)

config = {
    "accountID" : accountID,
    "apiKey" : apiKey
}

from ray.rllib.algorithms.callbacks import DefaultCallbacks

class InfoLog(DefaultCallbacks):
    def on_episode_step(self, *, worker, base_env, policies, episode, **kwargs):
        episode.custom_metrics["custom_reward"] = episode.total_reward
        print(f"Episode Reward: {episode.total_reward}")

from ray.rllib.algorithms.ppo import PPOConfig
ppoConfig = (
    PPOConfig()
    .environment(
        "Deriv12Env",
        env_config = config,  # `config` to pass to your env class
    )
    .callbacks(InfoLog)
    .api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)
    .debugging(log_level = "ERROR")
)

built = None
algo = ppoConfig.build(logger_creator=built)



# np.random.randint(0,5)
# env = fx.ForexApi(apiKey,accountID)
# data = fx.hr("EUR_USD")

# y = data.to_numpy()[0]

# g = deriv12({
#     "accountID" : accountID,
#     "apiKey" : apiKey
# })


c:\Users\excal\AppData\Local\Programs\Python\Python310\lib\site-packages\ray\rllib\algorithms\algorithm.py:569: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
c:\Users\excal\AppData\Local\Programs\Python\Python310\lib\site-packages\ray\tune\logger\unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
c:\Users\excal\AppData\Local\Programs\Python\Python310\lib\site-packages\ray\tune\logger\unified.py:53: RayDep

In [9]:
res = algo.train()
    

(RolloutWorker pid=20740) 2025-01-27 09:02:19,875	ERROR actor_manager.py:187 -- Worker exception caught during `apply()`: list index out of range
(RolloutWorker pid=20740) Traceback (most recent call last):
(RolloutWorker pid=20740)   File "c:\Users\excal\AppData\Local\Programs\Python\Python310\lib\site-packages\ray\rllib\utils\actor_manager.py", line 183, in apply
(RolloutWorker pid=20740)     return func(self, *args, **kwargs)
(RolloutWorker pid=20740)   File "c:\Users\excal\AppData\Local\Programs\Python\Python310\lib\site-packages\ray\rllib\execution\rollout_ops.py", line 108, in <lambda>
(RolloutWorker pid=20740)     (lambda w: w.sample(**random_action_kwargs))
(RolloutWorker pid=20740)   File "c:\Users\excal\AppData\Local\Programs\Python\Python310\lib\site-packages\ray\util\tracing\tracing_helper.py", line 467, in _resume_span
(RolloutWorker pid=20740)     return method(self, *_args, **_kwargs)
(RolloutWorker pid=20740)   File "c:\Users\excal\AppData\Local\Programs\Python\Python31

(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff9b866823215c6c27205df80d01000000 Worker ID: 2776fd0594aebb08931aebeac176f6a873d09b877dc76c3a784e8a30 Node ID: 01fafda64e3382878392cf563885250b87c49b03892e7e479865a271 Worker IP address: 127.0.0.1 Worker port: 55551 Worker PID: 20740 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly. Worker exits with an exit code 1.


2025-01-27 09:03:20,310	WARNING rollout_ops.py:121 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
2025-01-27 09:03:21,862	ERROR actor_manager.py:804 -- Ray error (The actor 5cdad7412495ee2659f8a25801000000 is unavailable: The actor is temporarily unavailable: IOError: The actor is restarting.. The task may or maynot have been executed on the actor.), taking actor 1 out of service.
2025-01-27 09:03:21,863	ERROR actor_manager.py:804 -- Ray error (The actor 9b866823215c6c27205df80d01000000 is unavailable: The actor is temporarily unavailable: IOError: The actor is restarting.. The task may or maynot have been executed on the actor.), taking actor 2 out of service.
2025-01-27 09:03:21,863	ERROR actor_manager.py:635 -- The actor 5cdad7412495ee2659f8a25801000000 is unavailable: The actor is temporarily unavailable: IOError: The actor 

(pid=18668) Stack (most recent call first):
(pid=18668)   File "<frozen importlib._bootstrap>", line 241 in _call_with_frames_removed
(pid=18668)   File "<frozen importlib._bootstrap_external>", line 1176 in create_module
(pid=18668)   File "<frozen importlib._bootstrap>", line 571 in module_from_spec
(pid=18668)   File "<frozen importlib._bootstrap>", line 674 in _load_unlocked
(pid=18668)   File "<frozen importlib._bootstrap>", line 1006 in _find_and_load_unlocked
(pid=18668)   File "<frozen importlib._bootstrap>", line 1027 in _find_and_load
(pid=18668)   File "c:\Users\excal\AppData\Local\Programs\Python\Python310\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 73 in <module>
(pid=18668)   File "<frozen importlib._bootstrap>", line 241 in _call_with_frames_removed
(pid=18668)   File "<frozen importlib._bootstrap_external>", line 883 in exec_module
(pid=18668)   File "<frozen importlib._bootstrap>", line 688 in _load_unlocked
(pid=18668)   File "<frozen importlib._bo

In [30]:
import forex as fx

a = fx.min("EUR_USD")


print(fx.to_date(a.data[0].iloc[-1,0]))
a = a.to_numpy()[0][:,2]

fx.algo.deriv12(a)


2025-01-28 16:19:00


-1